<a href="https://colab.research.google.com/github/ilya-myas/python-random-quote/blob/master/sprint1_prj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий ревьюера</h1>

Привет! Меня зовут Сысоева Вера.
    
Далее в файле мои комментарии ты сможешь найти в ячейках, аналогичных данной ( если рамки комментария зелёные - всё сделано правильно; жёлтые - есть замечания, но не критично; красные - нужно переделать). Не удаляй эти комментарии и постарайся учесть их в ходе выполнения проекта.
    
Я буду поддерживать версионность комментариев, и при следующих итерациях комментарии будут выделены, как "Комментраии ревьюера - 2 (3 и.т.д.)"
    
Если у тебя есть замечания, возражения и вопросы, ты можешь писать свои комментарии, по возможности выделяй их так же цветом (например, синим), чтоб их можно было легко отделить от текста проекта. 
    
Кстати, предлагаю сразу общаться на "ты".
    
Увидев у тебя ошибку, в первый раз я лишь укажу на ее наличие и дам тебе возможность самой найти и исправить ее. На реальной работе твой начальник будет поступать так же, а я пытаюсь подготовить тебя именно к работе аналитиком. Но если ты пока не справишься с такой задачей - при следующей проверке я дам более точную подсказку.

## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий ревьюера</h1>

Описание целей и задач проекта важно, не забывай про это в следующих проектах, где не будет такого подробного шаблона. План проекта тоже очень важен.
    
Также удобно вначале следать интерактивное оглавление.
    
Само оглавление делаешь по такому типу:  
1. [Шаг 1. Откроем файл с данными и изучим общую информацию](#step1)  
2. [Шаг 2](#step2)  

А в заголовках оставляешь метки вот так:  
### Шаг 1. Откроем файл с данными и изучим общую информацию <a id="step1"></a>   
    
    
    
Это сильно повышает привлекательность проекта.
</div>

### Содержание проекта <a id="step0"></a>

1. [Шаг 1. Откроем файл с данными и изучим общую информацию](#step1)  
2. [Шаг 2. Выполним предварительную обработку данных](#step2)  
2.1. [Обработаем пропуски](#step3)  
2.2. [Заменим тип данных](#step4)  
2.3. [Обработаем дубликаты](#step5)  
2.4. [Лемматизируем цели](#step6)  
2.5. [Категоризируем данные](#step7)  
3. [Шаг 3. Проанализируем интересующие нас данные и ответим на вопросы](#step8)  
4. [Шаг 4. Общий вывод](#step9)  

### Шаг 1. Откроем файл с данными и изучим общую информацию <a id="step1"></a>

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving data.csv to data.csv


In [ ]:
import pandas as pd
from nltk.stem import SnowballStemmer 
russian_stemmer = SnowballStemmer('russian')
from pymystem3 import Mystem
m = Mystem() 
from collections import Counter
#from google.colab import files
#uploaded = files.upload()
data = pd.read_csv('data.csv')
#загрузили файли в датафрейм, теперь будем изучать его.

In [ ]:
data.info()#посмотрим общую информацио о данных.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [ ]:
data.head(10)#посмотрим первые 10 строк.

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [ ]:
data.columns#проверим отсутствие лишних пробелов в названии столбцов.

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

In [ ]:
data.isnull().sum()#узнаем сколько нулевах значений в столбцах.

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [ ]:
#children_list = data['children'].unique()#Проверим сколько детей бывает у заёмщиков.
#children_list
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [ ]:
#dob_years_list = data['dob_years'].unique()#Проверим есть ли неправильно заполненные данные о возрасте.
#dob_years_list
data['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [ ]:
#education_list = data['education'].unique()#Проверим образование у заёмщиков.
#education_list
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

In [ ]:
#gender_list = data['gender'].unique()#Проверим пол.
#gender_list
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [ ]:
#family_status_list = data['family_status'].unique()#Проверим семейное положение.
#family_status_list
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [ ]:
#income_type_list = data['income_type'].unique()#Проверим кем работают заёмщики.
#income_type_list
data['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

In [ ]:
#debt_list = data['debt'].unique()#Проверим есть ли ошибки в графе задолжностей.
#debt_list
data['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

In [ ]:
#purpose_list = data['purpose'].unique()#Узнаем на какие цели были взяты кредиты.
#purpose_list
data['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

In [ ]:
data.duplicated().sum()#Проверим какое количество дублированных записей

54

Просмотр дубликатов <a id="step10"></a>

In [ ]:
data_dub = data[data.duplicated(keep = False)]#Просмотрим эти дублированные записи
data_dub.sort_values(by=['children', 'dob_years', 'education', 'family_status', 'gender', 'income_type', 'debt', 'purpose'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
15892,0,NaN,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,NaN,сделка с подержанным автомобилем
19321,0,NaN,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,NaN,сделка с подержанным автомобилем
3452,0,NaN,29,высшее,0,женат / замужем,0,M,сотрудник,0,NaN,покупка жилой недвижимости
18328,0,NaN,29,высшее,0,женат / замужем,0,M,сотрудник,0,NaN,покупка жилой недвижимости
520,0,NaN,35,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
9238,2,NaN,34,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,покупка жилья для сдачи
9013,2,NaN,36,высшее,0,женат / замужем,0,F,госслужащий,0,NaN,получение образования
14432,2,NaN,36,высшее,0,женат / замужем,0,F,госслужащий,0,NaN,получение образования
11033,2,NaN,39,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,сыграть свадьбу


### Вывод

Открыли файл, изучили его, выявили следующие неприятные моменты:
1. Много заёмщиков без указанного трудового стажа. Формат записи так же не понятен, похоже на то, что часть стажа записана днями и часть часами. Так как эти данные не критичны, заполним ячейки стажа без записи 0.
2. В столбце образование надо привести записи к единому регистру.
3. Интересная информация - есть отрицательное количество детей и 20 детей. Надо в дальнейшем будет проверить эти записи.
4. Есть заёмщики с возрастом 0 лет. В дальнейшем исправим их.
5. Есть заёмщики без пола. В дальнейшем исправим их.
6. Много заёмщиков без указанного дохода. В дальнейшем запишем среднее значение дохода для каждой категории заёмщиков.
7. Очень много похожих целей, но с разными вариантами написания. В дальнейшем приведём их единому виду. 
8. Есть дублированные данные в таблице. Далее их удалим, так как их количество не критично.

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий ревьюера</h1>

Молодец, ты отлично, очень подробно рассмотрел сырые данные. Здорово, что ты так подробно все описал, отличный план на предобработку получился.
</div>

### [Содержание](#step0) 

### Шаг 2. Выполним предварительную обработку данных <a id="step2"></a>

### Обработаем пропуски <a id="step3"></a>

In [ ]:
#children_20 = data.loc[data['children'] == 20]
#children_20
data['children'] = data['children'].abs()#.Считаем что -1 ошибка ввода, заменим на 1
data['children'] = data.replace(to_replace = 20, value = 2, regex = True) #Считаем, что 20 это ошибка ввода, заменим на 2
#data.loc[data['children'] == 20] = 2 #
#data.loc[data['children'] = 2  
children_counts = data['children'].value_counts() #Проверим результат
children_counts

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [ ]:
data['days_employed'] = data['days_employed'].fillna(0)#Заполняем ячейки стажа без записи 0.
data['days_employed'] = data['days_employed'].abs()#Избавляемся от отрицательных значений
data.isnull().sum()

children               0
days_employed          0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [ ]:
dob_years_mean = int(data['dob_years'].mean()) #Вычислим средний возраст и округлим его до целого значения
#dob_years_mean
data['dob_years'] = data['dob_years'].replace(to_replace = 0, value = dob_years_mean, regex = True)#Заменим 0-ой возраст на среднее занчение.
dob_years_counts = data['dob_years'].value_counts()
dob_years_counts

35    617
43    614
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [ ]:
for column in ['education','family_status' , 'income_type', 'purpose']:#Приведём все строковые столбцы к единому регистру.
    data[column] = data[column].str.lower()
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий ревьюера</h1>
Отличное использование цикла
</div>

In [ ]:
education_counts = data['education'].value_counts()
education_counts

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [ ]:
education_id_counts = data['education_id'].value_counts()
education_id_counts

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

In [ ]:
family_status_counts = data['family_status'].value_counts()
family_status_counts

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [ ]:
family_status_id_counts = data['family_status_id'].value_counts()
family_status_id_counts

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

In [ ]:
gender_counts = data['gender'].value_counts()#Посмотрим как берут кретит заёмщики разного пола
gender_counts

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [ ]:
gender_xna = data.loc[data['gender'] == 'XNA']#Посмотрим запись с неопределённым полом.
gender_xna

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


In [ ]:
#Создадим промежуточную таблицу, сгруппированую по типу занятости, для вычисления среднего дохода. 
median_table = data.groupby('income_type').agg({'total_income': 'median'})

#Заполним пропущенные значения средним значением для данной категории занятости.
for inc_type in median_table.index:
    inc_med = median_table.loc[inc_type, 'total_income']
    data.loc[data['income_type'] == inc_type, 'total_income'] = data.loc[data['income_type'] == inc_type, 'total_income'].fillna(inc_med)
data.isnull().sum()#Проверим остались ли не заполненные данные в таблице

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий ревьюера</h1>
Ок, замена на медиану по типу дохода - оптимальное решение
</div>

### Вывод

В первом шаге мы изучили таблицу. К записи данных в ней были выявлены замечания.
1. С графе "дети" были исправлены данные о количесте детей -1 и 20. Данные значения похоже на опечатку при вводе этих данных. Исправили на похожие по смыслу -1 на 1, а 20 на 2(при наборе на цифровом блоке клавиатуры явно была нажата ещё и цифра 0)
2. В графе "стаж" не заполненные данные замени на 0, так как эти данные на дальнейший анализ не влияют.
3. В графе "возраст" заменили нулевой врозраст на средний по таблице данных ,эти данные не критичны.
4. Графу образование привели к единому регистру нгаписания. Этим исключили дублирование данных об образовании.
5. В данных о поле заёмщиков есть одна запись с неуказанным полом. Анализ строки не позволяет выбрать замену, оставил как есть.
6. В графе "доход" пропуски заменил на среднее значение дохода для данной категории заёмщика.

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий ревьюера</h1>
Молодец, все пропуски и аномалии в данных обнаружили и обработали
</div>

### [Содержание](#step0) 

### Заменим тип данных <a id="step4"></a>

In [ ]:
data['days_employed'] = data['days_employed'].astype('int64')#Изменим тип данных в столбце стажа
data['total_income'] = data['total_income'].astype('int64')#Изменим тип данных в столбце дохода
data.info()#Проверим данные

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  object
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(6), object(6)
memory usage: 2.0+ MB


In [ ]:
data.head(10)#Посмтрим первые 10 строк

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


### Вывод

Для удобства дальнейших расчетов заменили вещественные числа в ячейках на целочисленные.

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий ревьюера</h1>
Да, типы данных оптимизированы. Это важный момент на практике, хотя на таком сравнительно небольшом наборе это может показаться незначитальным, но в реальности можно сильно сократить занимаемый объем данных
</div>

### [Содержание](#step0) 

### Обработаем дубликаты <a id="step5"></a>

In [ ]:
data = data.drop_duplicates().reset_index(drop=True)#Удалим дублированные данные
data.duplicated().sum()#Проверим дубликаты

0

### Вывод

Удалили обнаруженные в записях дубликаты. Дублированные данные об образовании исправили ранее.

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий ревьюера</h1>
Хорошо, дубликаты верно удалены.
    
Но все же стоило посмотреть сколько дублей у нас в наборе - это интересно. Плюс стоило поподробнее описать природу пропусков. Откуда они получились? Можно ли их безболезненно удалить?

<div style="border:solid blue 2px; padding: 14px"> <h1 style="color:blue; margin-bottom:14px">Комментарий студента-аналитика</h1>
Дубликаты были посчитаны и промотрены в шаге 1, при изучении данных в таблице. Сделал там вывод дубликатов более наглядней. 
</div>

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий ревьюера - 2</h1>
 
Хорошо. На таком не большом проекте это не очевидно, но в комплексных проектах иногда приходится повторятся. Просто иногда люди смотрят отдельные части и по оглпвлению переходят. Захотел человек (например Data Engineer) узнать что происходит с дублированием. А в этом разделе он видит просто 0.
    
При этом дублей на самом деле больше чем в первом разделе. При заменен регитра их количество увеличивается. Такое часто происходит и лучше еще раз все проверять непосредственно перед удалением.
</div>

[Посмотреть дубликаты](#step10) 


### [Содержание](#step0) 

### Лемматизируем цели <a id="step6"></a>

In [ ]:
data['purpose_lemma'] = data['purpose'].apply(m.lemmatize)#Все цели займа выделяем в отдельный столбец и лемматизиурем его
print(Counter(data['purpose_lemma'].sum()))#Смотрим сколько получилось уникальных слов

KeyboardInterrupt: ignored

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly&response_type=code

Enter your authorization code:
228916


ValueError: ignored

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий ревьюера</h1>
Отлично, использование counter - это правильно, список целей получен обосновано
</div>

In [ ]:
#Состаавляем функцию для дальнейшей обратобки с помощью ключевых слов
def purpose_category(lemmas):
    if 'свадьба' in lemmas:
        return 'Займ на свадьбу'
    if ('жилье' in lemmas) or ('недвижимость' in lemmas):
        return 'Займ на покупку недвижимости'
    if 'автомобиль' in lemmas:
        return 'Займ на покупку автомобиля'
    if 'образование' in lemmas:
        return 'Займ на образование'
    
data['purpose'] = data['purpose_lemma'].apply(purpose_category)#Заполняем цели изменнеными данными
purpose_counts = data['purpose'].value_counts()#Проверяем количество измененных данных 
purpose_counts

In [ ]:
data.head(10)#Смотрим как выглядят данный после всех обработок

### Вывод

В данных присутсвуют множестенные цели займа, но общим смыслом его получения. С помощью лемматизации по ключевым словам этих целей привели эти записи к единому виду для удобства дальнейшего анализа.

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий ревьюера</h1>
Отлично, лемматизация выполнена верно. Молодец, что разобрался с этой не простой темой
</div>

### [Содержание](#step0) 

### Категоризируем данные <a id="step7"></a>

In [ ]:
#Делаем промежуточную таблицу с выборкой по количеству детей и задожностью по кредиту
children_rezult = data.pivot_table(index = 'children', columns = 'debt', values = 'dob_years', 
                                   aggfunc = 'count', fill_value= 0)
children_rezult.columns = ['Выплачивают вовремя', 'Задерживают выплаты']
children_rezult = children_rezult.astype('int64')

children_rezult['Доля должников'] = (children_rezult['Задерживают выплаты']/
                                (children_rezult['Задерживают выплаты'] 
                                 + children_rezult['Выплачивают вовремя'])).map('{:.2%}'.format)

In [ ]:
#Делаем промежуточную таблицу с выборкой по семейному положению и задожностью по кредиту
family_status_rezult = data.pivot_table(index = 'family_status', columns = 'debt', values = 'dob_years', 
                                        aggfunc = 'count', fill_value= 0)
family_status_rezult.columns = ['Выплачивают вовремя', 'Задерживают выплаты']
family_status_rezult = family_status_rezult.astype('int64')

family_status_rezult['Доля должников'] = (family_status_rezult['Задерживают выплаты']/
                                (family_status_rezult['Задерживают выплаты'] 
                                 + family_status_rezult['Выплачивают вовремя'])).map('{:.2%}'.format)

In [ ]:
#Делаем промежуточный столбец таблицы с категориями по уровню дохода
def sort_total_income(income):
    if income < data['total_income'].quantile(0.25):
        return 'Минимальный доход'
    if income < data['total_income'].quantile(0.50):
        return 'Средний доход'
    if income < data['total_income'].quantile(0.75):
        return 'Повышеный доход'
    return 'Высокий доход'
#добавляем в таблицу столбец с категориями по доходам
data['true_income'] = data['total_income'].apply(sort_total_income)
true_income_counts = data['true_income'].value_counts()
true_income_counts

In [ ]:
#Делаем промежуточную таблицу с выборкой по уровню дохода и задожностью по кредиту
total_income_rezult = data.pivot_table(index = 'true_income', columns = 'debt', values = 'dob_years', 
                                       aggfunc = 'count', fill_value= 0)
total_income_rezult.columns = ['Выплачивают вовремя', 'Задерживают выплаты']
total_income_rezult = total_income_rezult.astype('int64')

total_income_rezult['Доля должников'] = (total_income_rezult['Задерживают выплаты']/
                                (total_income_rezult['Задерживают выплаты'] 
                                 + total_income_rezult['Выплачивают вовремя'])).map('{:.2%}'.format)

In [ ]:
#Делаем промежуточную таблицу с выборкой по цели займа и задожностью по кредиту
purpose_rezult = data.pivot_table(index = 'purpose', columns = 'debt', values = 'dob_years', 
                                  aggfunc = 'count', fill_value= 0)
purpose_rezult.columns = ['Выплачивают вовремя', 'Задерживают выплаты']
purpose_rezult = purpose_rezult.astype('int64')

purpose_rezult['Доля должников'] = (purpose_rezult['Задерживают выплаты']/
                                (purpose_rezult['Задерживают выплаты'] 
                                 + purpose_rezult['Выплачивают вовремя'])).map('{:.2%}'.format)

In [ ]:
#Делаем промежуточную таблицу с выборкой по всем целям задания и задожностью по кредиту
#При желании можем комбинировать нужные нам категории для анализа
final_rezult = data.pivot_table(index = ['children', 'family_status', 'true_income', 'purpose'], 
                                columns = 'debt', values = 'dob_years', aggfunc = 'count', fill_value= 0)
final_rezult.columns = ['Выплачивают вовремя', 'Задерживают выплаты']
final_rezult = final_rezult.astype('int64')

final_rezult['Доля должников'] = (final_rezult['Задерживают выплаты']/
                                (final_rezult['Задерживают выплаты'] 
                                 + final_rezult['Выплачивают вовремя'])).map('{:.2%}'.format)

### Вывод

Создали промежуточные таблицы для дальнейшего изучения интересующих нас категорий заёмщиков.

<div style="border:solid red 2px; padding: 20px"> <h1 style="color:red; margin-bottom:20px">Комментарий ревьюера</h1>
Ой, нет вот так делать не надо. Создавать под каждый вопрос два набора - это оооочень ресурсозатратно.
    
    
В категоризации нужно выделить просто категории по целям - ты это сделал в предыдущем разделе, по уровню дохода - ты верно написал функцию и возможно по количеству детей.
    
В вот рассчеты нужно делать с использованием сводных таблиц и все нужно получить из основного набора
    
<div style="border:solid blue 2px; padding: 14px"> <h1 style="color:blue; margin-bottom:14px">Комментарий студента-аналитика</h1>
Согласен, было сделано через одно место. На тот момент соображал уже туго, поэтому сделал как пришло в голову.
Исправил всё по нормальному, плюс добавил сводную категоризацию по всем нужных параметрам.
   
</div>

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий ревьюера - 2</h1>
 
Отлично, теперь все верно, молодец, что разобрался.
</div>

### [Содержание](#step0) 

### Шаг 3. Проанализируем интересующие нас данные и ответим на вопросы <a id="step8"></a>

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
children_rezult#Изучим связь задержки выплат кредита с количеством детей

### Вывод

Удивительно, но заёмщики с 5 детьми самые ответственые, задержек выплат нет.
Заёмщики с 1, 2 и 4 детьми задерживают выплаты в среднем 9.5%.
Заёмщики с 3 детьми задерживают выплаты в среднем 8.2%.
Заёмщики без детей задерживаю выплаты меньше остальных, примерно 7,5%

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
family_status_rezult#Изучим связь задержки выплат кредита с семейным статусом заёмщика

### Вывод

Самые не надёжные заемщики, это люди не бывшие в семейных отношениях и живущие без регистрации брака, более 9%. Думаю из-за того, что нету связанной финансово второй половины семьи. Чуть меньше задержек выплат у семейных и разведенных заёмщиков. Эти люди более рационально планируют свои расходы. Люди, потерявшие часть своей семьи, более отвественны.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
total_income_rezult#Изучим связь задержки выплат кредита с уровнем дохода

### Вывод

Как мы заметили из таблицы самые ненадежные заёмщики, это люди со средним и повышеным доходом, в среднем 8.7% задерживают платежи . Удивительно, но заёмщики с высоким доходом так же задерживаю платежи, 7.1%. Заёмщики с минимальным доходом по количеству задержек занимают середину, около 8%.

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
purpose_rezult#Изучим связь задержки выплат кредита с целью займа

### Вывод

Больше всего задолжников в займах на автомобиль. Думаю, так как он взят в кредит, приобрели модель подороже и не расчитали возможность выплачивать повышеный платеж. В займах на образование задержки скорее всего связаны с самим образованием. Люди, которые учаться, подрабатывают. Или они учаться заочно, и на время заочной учёбы у них просадка по ЗП. Займ на свадьбу чуть надежней, но все равно процент задержек платежей 8%. Самый выгодный для финансовой организации займ на недвижимость, здесь меньше всего процент задержки. Наверное покупка недвижимости самый обдуманный вид займа.

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий ревьюера</h1>
Ок, подход правильный, но реулизация не верная. Пересмотри техически рассчет вероятности невозврата
</div>

- Выведем сводную таблицу по всем четырём категориям и изучим её

In [ ]:
#Для более детального анализа можем изучить сводную таблицу по всем четырём категориям задания
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(final_rezult)

### [Содержание](#step0) 

### Шаг 4. Общий вывод <a id="step9"></a>

1. В любой категории заёмщиков есть должники. На задержку выплат оказывает влияние наличие детей, их наличие негативно отражается на выплатах, в среднем около 9%. Есть исключение - это заёмщики с 5 детьми. Они выплючиват вовремя, но так как таких заёмщиков минимально, всего 9, в общий расчет их не стоит принимать. 
2. Вызывают опасения не семейные заёмщики, более 9% задержек платежей. Семейные и в разводе на 1.5% меньше задерживают выплаты.
3. Не зависимо от дохода, всё равно есть задерживающие платежи. При этом, заёмщики с высоким и минимальным доходом более отвественны, менее 8% задедерживающих платежи. Среди заёмщиков со средним и повышенным доходом задерживаюх плетежи уже 8.7%.
4. Самые рискованные категории займов , это займы на автомобиль и образование, более 9%, Следующие, это молодожёны, их среди должников 8%. Займы на недвижимость самые популяные, среди них должников меньше всего, 7.2%.

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Общий комментарий по проекту</h1>

Ты хорошо поработал над проектом, молодец. Сам анализ правильный, немного рапутался с методом рассчета, но это поправимо.
    
Ты отлично работаешь с данными и уже освоил много полезных методов, здорово. 
    
В целом ты молодец, отлично справляешься с программой, критических замечаний по проекту нет. Только постарайся делать именно рассказ по данным, связный рассказ с итогами и выводами. 
    
Жду улучшенный вариант проекта. Удачи
</div>

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий ревьюера - 2</h1>
 
Проект отлично доработан, молодец.
</div>

### [Содержание](#step0) 

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.